<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/AI/NeqSim_and_Seeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeqSim and Data Analytics using Seeq

https://www.seeq.com/

<figure>
<img src='https://github.com/EvenSol/NeqSim-Colab/blob/master/figures/neqsimseeq.png?raw=true', width="300", height="300"
</figure>

### Bridging Process Data Analytics and Thermodynamic Simulation

The integration of Seeq, an advanced analytics platform for process  data, with NeqSim, unlocks significant new capabilities for engineers and data scientists. This combination allows for the creation of sophisticated "soft sensors" and first-principle models that can analyze, predict, and optimize industrial processes with greater accuracy.

#### What is Seeq?

Seeq is a software application designed for the rapid investigation and analysis of time-series data from industrial processes. It connects to various data historians like OSIsoft PI and Honeywell PHD, as well as relational databases, allowing users to access and analyze their operational data without needing to move or copy it. Seeq empowers subject matter experts to cleanse, contextualize, visualize, and model their data to improve production outcomes such as yield, quality, and availability. Its intuitive, web-based interface is designed for self-service analytics, enabling engineers to perform diagnostic, monitoring, and predictive analyses without requiring coding experience.

#### Advantages and Possibilities of NeqSim Integration

Integrating NeqSim's first-principles thermodynamic and process models into Seeq's data-rich environment offers several key advantages:

*   **Creation of Virtual Sensors:** By feeding live process data (like temperature, pressure, and composition) from Seeq into NeqSim models, you can calculate unmeasured fluid properties in real-time. This allows you to create "soft sensors" for critical parameters that are difficult or expensive to measure directly, such as hydrate formation temperature, phase envelopes, dew points, or the density and viscosity of a multiphase fluid.

*   **Physics-Based Analytics:** The integration moves beyond purely data-driven models (machine learning) by incorporating fundamental chemical engineering principles. This results in more robust and reliable models that perform well even outside of normal operating conditions. NeqSim can calculate complex behaviors like hydrate formation, multiphase flow, and chemical reactions, providing a deeper understanding of the process.

*   **Enhanced Process Monitoring and Optimization:** You can compare the actual performance of a unit operation (captured in Seeq) against its theoretical performance calculated by a NeqSim model. This is invaluable for monitoring equipment efficiency, identifying operational issues, and optimizing processes like separation, compression, or dehydration.

*   **Proactive Problem Solving:** By simulating "what-if" scenarios, engineers can use the integrated platform to proactively address potential issues. For instance, you can use real-time data to predict the likelihood of hydrate formation in a pipeline as operating conditions change, allowing for preventative action.

*   **Improved Model Development:** Seeq's tools can be used to easily identify and cleanse relevant periods of historical data (e.g., steady-state operations) to be used for tuning and validating NeqSim's thermodynamic models, ensuring the simulations accurately reflect the real-world process.

This synergy enables a more profound and accurate understanding of process behavior, leading to improved decision-making, enhanced operational efficiency, and proactive process management.

# Installation of NeqSim in Seeq datalab
The following script demonstrates how to install neqsim in seeq datalab. We install neqsin via pip install neqsim and need a Java development environment to be abble to run neqsim.

In [ ]:
!pip install neqsim

In [ ]:
#run this only first time in workbook
import os
import jdk
java_path = jdk.install('21')
os.environ["JAVA_HOME"] = java_path
os.environ["PATH"] = java_path + "/bin:" + os.environ["PATH"]

### Integrating NeqSim with Seeq Data Lab for Advanced Process Simulations

Creating a Datalab file that utilizes NeqSim involves installing the NeqSim Python package within the Seeq Data Lab environment, pulling data from Seeq, performing simulations with NeqSim, and then pushing the results back to Seeq for visualization and further analysis.

#### Key Steps for Integration:

1.  **Installation of NeqSim:** The `neqsim` Python package can be installed in a Seeq Data Lab notebook using a `pip` command. It is important to note that NeqSim has a Java dependency, which should be handled by the Seeq Data Lab environment.
2.  **Data Retrieval from Seeq:** The `seeq` library, specifically the `spy` module, is used to connect to the Seeq server and pull in the necessary process data, such as temperatures, pressures, and flow rates, into a Pandas DataFrame.
3.  **Process Calculations with NeqSim:** With the process data available in the Datalab notebook, you can utilize the NeqSim library to perform a wide range of calculations. This can include phase equilibrium calculations, fluid property estimations, and process simulations. NeqSim's Python interface allows for the creation of fluid objects and the execution of various thermodynamic operations.
4.  **Pushing Results back to Seeq:** After the NeqSim calculations are complete, the resulting data can be pushed back to Seeq as new signals. This is also accomplished using the `spy` library, making the simulation results available for visualization and analysis within the Seeq Workbench.

#### Example Workflow in a Seeq Data Lab Notebook:

Below is a conceptual example of a Python script within a Seeq Data Lab notebook that demonstrates the integration of NeqSim and Seeq.

```python
# Step 1: Install NeqSim
!pip install neqsim

# Step 2: Import necessary libraries
from seeq import spy
import pandas as pd
from neqsim.thermo import fluid, TPflash

# Step 3: Log in to Seeq (if not already logged in through the Data Lab interface)
# spy.login(url='https://your-seeq-server.com', credentials_file='path/to/credentials.key')

# Step 4: Search for and pull process data from Seeq
# Define the time range for the analysis
end_time = spy.utils.now()
start_time = end_time - pd.Timedelta(days=7)

# Search for relevant signals (e.g., pressure, temperature, and component molar fractions)
process_signals = spy.search({
    'Path': 'Path to your asset',
    'Name': ['Pressure', 'Temperature', 'Methane Molar Fraction', 'Ethane Molar Fraction']
})

# Pull the data into a Pandas DataFrame
process_data = spy.pull(process_signals, start=start_time, end=end_time, grid='1h')

# Step 5: Perform NeqSim calculations
# For each timestamp in the process data, run a thermodynamic calculation
results = []
for index, row in process_data.iterrows():
    # Create a fluid object in NeqSim with data from Seeq
    fluid_components = {
        "methane": row['Methane Molar Fraction'],
        "ethane": row['Ethane Molar Fraction'],
        # Add other components as needed
    }
    
    # Check if component data is valid
    if all(pd.notna(value) for value in fluid_components.values()):
        try:
            fluid1 = fluid.createFluid(fluid_components)
            fluid1.setTemperature(row['Temperature'], "K")
            fluid1.setPressure(row['Pressure'], "bara")
            
            # Perform a TPflash calculation
            TPflash(fluid1)
            
            # Extract a calculated property (e.g., density)
            density = fluid1.getPhase("gas").getDensity("kg/m3")
            results.append({'Timestamp': index, 'Calculated Density': density})
        except Exception as e:
            print(f"Error during NeqSim calculation at {index}: {e}")
            results.append({'Timestamp': index, 'Calculated Density': None})
    else:
        results.append({'Timestamp': index, 'Calculated Density': None})


# Create a DataFrame from the results
results_df = pd.DataFrame(results)
results_df.set_index('Timestamp', inplace=True)

# Step 6: Push the calculated data back to Seeq
spy.push(data=results_df,
         metadata = {'Name': 'Calculated Gas Density',
                     'Description': 'Gas density calculated by NeqSim',
                     'Data ID': 'Calculated_Gas_Density_from_NeqSim'},
         workbook='Path to your Seeq Workbook',
         worksheet='Analysis Worksheet')
```

This example outlines a basic workflow. The specific NeqSim calculations and the Seeq data used can be adapted to a wide variety of process engineering challenges, such as:

*   **Hydrate formation prediction:** Using process conditions from Seeq to predict the risk of hydrate formation in pipelines.
*   **Phase envelope generation:** Creating phase envelopes for hydrocarbon mixtures based on real-time compositional data.
*   **Equipment performance monitoring:** Calculating theoretical performance of equipment like compressors or separators and comparing it with actual performance data from Seeq.

# Example of use of NeqSim with Seeq

We start by creating a process model with neqsim as in example:

https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/comparesimulations.ipynb

In [ ]:
import json
from typing import Optional, List

import pandas as pd
from pydantic.dataclasses import dataclass
from pydantic import Field, field_validator

# Import or reference the neqsim library
# import neqsim  # Uncomment this if needed, depending on your environment.

@dataclass
class ProcessInput:
    """
    A data class to define input parameters for an oil and gas process simulation
    using the NEQSim process modeling library.

    This class uses Pydantic to structure and validate the input parameters
    required for running the simulation. It ensures that the input data is of
    the correct type and within acceptable ranges.

    Attributes:
        feed_rate (float): Molar flow rate [kgmole/hr].
        molar_composition (List[float]): Molar composition of well fluid [mole fraction].
            Must contain exactly 17 components.
        Psep1 (float): Pressure of first stage separator [barg].
        Tsep1 (float): Temperature of first stage separator [°C].
        Psep2 (float): Pressure of second stage separator [barg].
        Tsep2 (float): Temperature of second stage separator [°C].
        Psep3 (float): Pressure of third stage separator [barg].
        Tsep3 (float): Temperature of third stage separator [°C].
        Tscrub1 (float): Temperature of first stage recompressor scrubber [°C].
        Tscrub2 (float): Temperature of second stage recompressor scrubber [°C].
        Tscrub3 (float): Temperature of third stage recompressor scrubber [°C].
        Tscrub4 (float): Temperature of fourth stage recompressor scrubber [°C].
        Pcomp1 (float): Pressure after 1st stage export compressor [barg].
        Trefig (float): Temperature after cooling export gas [°C].
        P_oil_export (float): Pressure after oil export pump [barg].
        T_oil_export (float): Temperature of export oil [°C].
        P_gas_export (float): Pressure of export gas [barg].
        T_gas_export (float): Temperature of export gas [°C].
        dP_20_HA_01 (float): Pressure drop over heater 20-HA-01 [bar].
        dP_20_HA_02 (float): Pressure drop over heater 20-HA-02 [bar].
        dP_20_HA_03 (float): Pressure drop over heater 20-HA-03 [bar].
        dP_21_HA_01 (float): Pressure drop over heater 21-HA-01 [bar].
        dP_23_HA_01 (float): Pressure drop over heater 23-HA-01 [bar].
        dP_23_HA_02 (float): Pressure drop over heater 23-HA-02 [bar].
        dP_23_HA_03 (float): Pressure drop over heater 23-HA-03 [bar].
        dP_24_HA_01 (float): Pressure drop over heater 24-HA-01 [bar].
        dP_25_HA_01 (float): Pressure drop over heater 25-HA-01 [bar].
        dP_25_HA_02 (float): Pressure drop over heater 25-HA-02 [bar].
        dP_27_HA_01 (float): Pressure drop over heater 27-HA-01 [bar].
    """

    feed_rate: float = Field(title="molar flow rate [kgmole/hr]")
    molar_composition: List[float] = Field(title="molar composition of well fluid [molefraction]")
    Psep1: float = Field(ge=0.0, le=100.0, title="Pressure of first stage separator [barg]")
    Tsep1: float = Field(ge=0.0, le=100.0, title="Temperature of first stage separator [C]")
    Psep2: float = Field(ge=0.0, le=100.0, title="Pressure of second stage separator [barg]")
    Tsep2: float = Field(ge=0.0, le=100.0, title="Temperature of second stage separator [C]")
    Psep3: float = Field(ge=0.0, le=100.0, title="Pressure of third stage separator [barg]")
    Tsep3: float = Field(ge=0.0, le=100.0, title="Temperature of third stage separator [C]")
    Tscrub1: float = Field(ge=0.0, le=100.0, title="Temperature of first stage recompressor scrubber [C]")
    Tscrub2: float = Field(ge=0.0, le=100.0, title="Temperature of second stage recompressor scrubber [C]")
    Tscrub3: float = Field(ge=0.0, le=100.0, title="Temperature of third stage recompressor scrubber [C]")
    Tscrub4: float = Field(ge=0.0, le=100.0, title="Temperature of fourth stage recompressor scrubber [C]")
    Pcomp1: float = Field(ge=0.0, le=100.0, title="Pressure after 1st stage export compressor [barg]")
    Trefig: float = Field(ge=0.0, le=100.0, title="Temperature after cooling export gas [C]")
    P_oil_export: float = Field(ge=0.0, le=200.0, title="Pressure after oil export pump [barg]")
    T_oil_export: float = Field(ge=0.0, le=100.0, title="Temperature of export oil [C]")
    P_gas_export: float = Field(ge=0.0, le=300.0, title="Pressure of export gas [barg]")
    T_gas_export: float = Field(ge=0.0, le=100.0, title="Temperature of export gas [C]")
    dP_20_HA_01: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 20-HA-01 [bar]")
    dP_20_HA_02: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 20-HA-02 [bar]")
    dP_20_HA_03: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 20-HA-03 [bar]")
    dP_21_HA_01: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 21-HA-01 [bar]")
    dP_23_HA_01: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 23-HA-01 [bar]")
    dP_23_HA_02: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 23-HA-02 [bar]")
    dP_23_HA_03: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 23-HA-03 [bar]")
    dP_24_HA_01: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 24-HA-01 [bar]")
    dP_25_HA_01: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 25-HA-01 [bar]")
    dP_25_HA_02: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 25-HA-02 [bar]")
    dP_27_HA_01: float = Field(ge=0.0, le=10.0, title="Pressure drop over heater 27-HA-01 [bar]")

    @field_validator("molar_composition")
    def check_mole_rates(cls, v):
        """
        Validates that the molar composition list has exactly 17 components.
        """
        if len(v) != 17:
            raise ValueError("Molar composition should have 17 components.")
        return v


@dataclass
class ProcessOutput:
    """
    A data class to define output results from an oil and gas process simulation.

    Attributes:
        mass_balance (Optional[float]): The mass balance result (percentage).
        results (Optional[dict]): A dictionary containing simulation results
            retrieved from the NEQSim process model JSON report.
    """
    mass_balance: Optional[float] = None
    results: Optional[dict] = None


def getprocess():
    """
    Create and return a NEQSim oil process object using default or
    dummy fluid/stream components. This function demonstrates the
    general structure of a multi-stage oil and gas separation process
    with scrubbing, compressing, cooling, and pumping steps.

    Returns:
        A NEQSim ProcessSystem object representing the oil and gas
        separation process.
    """
    # NOTE:
    #   The following code presupposes that `wellfluid` and `neqsim.process`
    #   are available in your environment. If they are not, you will need
    #   to provide them or modify references accordingly.
    #   The lines below are placeholders for demonstration.

    # Create a reference fluid (ensure 'wellfluid' is defined or created externally)
    wellstream = neqsim.process.equipment.stream.Stream("well stream", wellfluid)
    wellstream.setTemperature(60.0, 'C')
    wellstream.setPressure(33.01, 'bara')

    # Heater
    well_stream_cooler = neqsim.process.equipment.heatexchanger.Heater("20-HA-01", wellstream)

    # First-stage separator
    first_stage_separator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "20-VA-01", well_stream_cooler.getOutStream()
    )

    # Throttling valve from first-stage separator oil out
    oilvalve1 = neqsim.process.equipment.valve.ThrottlingValve(
        "VLV-100", first_stage_separator.getOilOutStream()
    )

    # Mixer for second-stage inlet
    oil_2nd_stage_mixer = neqsim.process.equipment.mixer.Mixer("MIX-101")
    oil_2nd_stage_mixer.addStream(oilvalve1.getOutStream())

    # Heater
    oilHeaterFromFirstStage = neqsim.process.equipment.heatexchanger.Heater(
        "20-HA-02", oil_2nd_stage_mixer.getOutStream()
    )

    # Second-stage separator
    seccond_stage_separator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "20-VA-02", oilHeaterFromFirstStage.getOutStream()
    )

    # Throttling valve from second-stage separator oil out
    oilvalve2 = neqsim.process.equipment.valve.ThrottlingValve(
        "VLV-102", seccond_stage_separator.getOilOutStream()
    )

    # Reflux from well stream (for demonstration, setFlowRate is trivial)
    oilreflux = wellstream.clone()
    oilreflux.setName("third stage reflux")
    oilreflux.setFlowRate(1e-6, 'kg/hr')

    # Mixer for third-stage inlet
    thirdstageoilmixer = neqsim.process.equipment.mixer.Mixer("MIX-102")
    thirdstageoilmixer.addStream(oilvalve2.getOutStream())
    thirdstageoilmixer.addStream(oilreflux)

    # Heater
    oilHeaterFromSeccondStage = neqsim.process.equipment.heatexchanger.Heater(
        "20-HA-03", thirdstageoilmixer.getOutletStream()
    )

    # Third-stage separator
    third_stage_separator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "20-VA-03", oilHeaterFromSeccondStage.getOutStream()
    )

    # Cooler
    firstStageCooler = neqsim.process.equipment.heatexchanger.Cooler(
        "23-HA-03", third_stage_separator.getGasOutStream()
    )

    # Scrubber
    firstStageScrubber = neqsim.process.equipment.separator.Separator(
        "23-VG-03", firstStageCooler.getOutStream()
    )

    # Pump
    firststagescrubberpump = neqsim.process.equipment.pump.Pump(
        "23-PA-01", firstStageScrubber.getLiquidOutStream()
    )

    # Recycle
    lp_resycle = neqsim.process.equipment.util.Recycle("LP oil resycle")
    lp_resycle.addStream(firststagescrubberpump.getOutStream())
    lp_resycle.setOutletStream(oilreflux)
    lp_resycle.setTolerance(1e-6)

    # First-stage recompressor
    first_stage_recompressor = neqsim.process.equipment.compressor.Compressor(
        "23-KA-03", firstStageScrubber.getGasOutStream()
    )
    first_stage_recompressor.setIsentropicEfficiency(0.75)

    # Mixer combining first-stage recompressor gas and second-stage separator gas
    firststagegasmixer = neqsim.process.equipment.mixer.Mixer("MIX-103")
    firststagegasmixer.addStream(first_stage_recompressor.getOutStream())
    firststagegasmixer.addStream(seccond_stage_separator.getGasOutStream())

    # Second-stage cooler
    seccond_stage_cooler = neqsim.process.equipment.heatexchanger.Cooler(
        "23-HA-02", firststagegasmixer.getOutStream()
    )

    # Second-stage scrubber
    seccond_stage_scrubber = neqsim.process.equipment.separator.Separator(
        "23-VG-02", seccond_stage_cooler.getOutStream()
    )

    # Add second-stage scrubber liquid to third-stage oil mixer
    thirdstageoilmixer.addStream(seccond_stage_scrubber.getLiquidOutStream())

    # Second-stage recompressor
    seccond_stage_recompressor = neqsim.process.equipment.compressor.Compressor(
        "23-KA-02", seccond_stage_scrubber.getGasOutStream()
    )
    seccond_stage_recompressor.setIsentropicEfficiency(0.75)

    # Mixer combining second-stage recompressor gas and first-stage separator gas
    exportgasmixer = neqsim.process.equipment.mixer.Mixer("MIX-100")
    exportgasmixer.addStream(seccond_stage_recompressor.getOutStream())
    exportgasmixer.addStream(first_stage_separator.getGasOutStream())

    # Dew point cooler
    dew_point_cooler = neqsim.process.equipment.heatexchanger.Cooler(
        "23-HA-01", exportgasmixer.getOutStream()
    )

    # Dew point scrubber
    dew_point_scrubber = neqsim.process.equipment.separator.Separator(
        "23-VG-01", dew_point_cooler.getOutStream()
    )

    # Add dew point scrubber liquid to second-stage mixer
    oil_2nd_stage_mixer.addStream(dew_point_scrubber.getLiquidOutStream())

    # First stage export compressor
    first_stage_export_compressor = neqsim.process.equipment.compressor.Compressor(
        "23-KA-01", dew_point_scrubber.getGasOutStream()
    )
    first_stage_export_compressor.setIsentropicEfficiency(0.75)

    # Cooler after first stage export compressor
    dew_point_cooler2 = neqsim.process.equipment.heatexchanger.Cooler(
        "24-HA-01", first_stage_export_compressor.getOutStream()
    )

    # Scrubber after second dew point cooler
    dew_point_scrubber2 = neqsim.process.equipment.separator.Separator(
        "24-VG-01", dew_point_cooler2.getOutStream()
    )

    # Add the liquid to second-stage mixer
    oil_2nd_stage_mixer.addStream(dew_point_scrubber2.getLiquidOutStream())

    # Gas splitter
    gas_splitter = neqsim.process.equipment.splitter.Splitter(
        'splitter', dew_point_scrubber2.getGasOutStream()
    )
    gas_splitter.setSplitNumber(2)
    gas_splitter.setFlowRates([-1, 2966.0], "kg/hr")  # Example usage

    # Fuel gas stream
    fuel_gas = gas_splitter.getSplitStream(1)
    fuel_gas.setName('fuel gas')

    # Heat exchanger
    gas_heatexchanger = neqsim.process.equipment.heatexchanger.HeatExchanger(
        "25-HA-01", gas_splitter.getSplitStream(0)
    )
    gas_heatexchanger.setGuessOutTemperature(273.15 + 15.0)
    gas_heatexchanger.setUAvalue(800e3)

    # Dew point cooler #3
    dew_point_cooler3 = neqsim.process.equipment.heatexchanger.Cooler(
        "25-HA-02", gas_heatexchanger.getOutStream(0)
    )

    # Dew point scrubber #3
    dew_point_scrubber3 = neqsim.process.equipment.separator.Separator(
        "25-VG-01", dew_point_cooler3.getOutStream()
    )

    # Add dew point scrubber #3 liquid to the export gas mixer
    exportgasmixer.addStream(dew_point_scrubber3.getLiquidOutStream())

    # Set feed for the heat exchanger
    gas_heatexchanger.setFeedStream(1, dew_point_scrubber3.getGasOutStream())

    # Second-stage export compressor
    seccond_stage_export_compressor = neqsim.process.equipment.compressor.Compressor(
        "27-KA-01", gas_heatexchanger.getOutStream(1)
    )
    seccond_stage_export_compressor.setIsentropicEfficiency(0.75)

    # Cooler after second-stage export compressor
    export_compressor_cooler = neqsim.process.equipment.heatexchanger.Cooler(
        "27-HA-01", seccond_stage_export_compressor.getOutStream()
    )

    # Final export gas
    export_gas = export_compressor_cooler.getOutStream()
    export_gas.setName('export gas')

    # Export oil cooler
    export_oil_cooler = neqsim.process.equipment.heatexchanger.Cooler(
        "21-HA-01", third_stage_separator.getOilOutStream()
    )

    # Export oil pump
    export_oil_pump = neqsim.process.equipment.pump.Pump(
        "21-PA-01", export_oil_cooler.getOutStream()
    )
    export_oil = export_oil_pump.getOutStream()
    export_oil.setName('export oil')

    # Create a process system
    oilprocess = neqsim.process.processmodel.ProcessSystem()
    oilprocess.add(wellstream)
    oilprocess.add(well_stream_cooler)
    oilprocess.add(first_stage_separator)
    oilprocess.add(oilvalve1)
    oilprocess.add(oil_2nd_stage_mixer)
    oilprocess.add(oilHeaterFromFirstStage)
    oilprocess.add(seccond_stage_separator)
    oilprocess.add(oilvalve2)
    oilprocess.add(oilreflux)
    oilprocess.add(thirdstageoilmixer)
    oilprocess.add(oilHeaterFromSeccondStage)
    oilprocess.add(third_stage_separator)
    oilprocess.add(firstStageCooler)
    oilprocess.add(firstStageScrubber)
    oilprocess.add(firststagescrubberpump)
    oilprocess.add(lp_resycle)
    oilprocess.add(first_stage_recompressor)
    oilprocess.add(firststagegasmixer)
    oilprocess.add(seccond_stage_cooler)
    oilprocess.add(seccond_stage_scrubber)
    oilprocess.add(seccond_stage_recompressor)
    oilprocess.add(exportgasmixer)
    oilprocess.add(dew_point_cooler)
    oilprocess.add(dew_point_scrubber)
    oilprocess.add(first_stage_export_compressor)
    oilprocess.add(dew_point_cooler2)
    oilprocess.add(dew_point_scrubber2)
    oilprocess.add(gas_splitter)
    oilprocess.add(gas_heatexchanger)
    oilprocess.add(dew_point_cooler3)
    oilprocess.add(dew_point_scrubber3)
    oilprocess.add(seccond_stage_export_compressor)
    oilprocess.add(export_compressor_cooler)
    oilprocess.add(export_oil_cooler)
    oilprocess.add(export_oil_pump)
    oilprocess.add(export_gas)
    oilprocess.add(export_oil)
    oilprocess.add(fuel_gas)

    return oilprocess


def updateinput(process, locinput):
    """
    Update the NEQSim process model with the provided input parameters.

    Attempts to set or update the operating conditions for each unit
    in the process. If a unit does not exist or is not named as expected,
    an AttributeError is caught and printed.

    Args:
        process: A NEQSim ProcessSystem object.
        locinput (ProcessInput): A validated ProcessInput object
            containing simulation parameters.
    """
    try:
        # Well stream
        process.getUnit('well stream').setFlowRate(locinput.feed_rate * 1e3 / 3600, 'mol/sec')
        process.getUnit('well stream').getFluid().setMolarComposition(locinput.molar_composition)
        process.getUnit('well stream').setPressure(locinput.Psep1 + locinput.dP_20_HA_01, "barg")
        process.getUnit('well stream').setTemperature(60.0, "C")

        # 20-HA-01
        process.getUnit('20-HA-01').setOutTemperature(locinput.Tsep1, "C")
        process.getUnit('20-HA-01').setOutPressure(locinput.Psep1, "barg")

        # VLV-100
        process.getUnit('VLV-100').setOutletPressure(locinput.Psep2 + locinput.dP_20_HA_02, 'barg')

        # 20-HA-02
        process.getUnit('20-HA-02').setOutTemperature(locinput.Tsep2, 'C')
        process.getUnit('20-HA-02').setOutPressure(locinput.Psep2, 'barg')

        # VLV-102
        process.getUnit('VLV-102').setOutletPressure(locinput.Psep3 + locinput.dP_20_HA_03, 'barg')

        # 20-HA-03
        process.getUnit('20-HA-03').setOutTemperature(locinput.Tsep3, 'C')
        process.getUnit('20-HA-03').setOutPressure(locinput.Psep3, 'barg')

        # 23-HA-03
        process.getUnit('23-HA-03').setOutTemperature(locinput.Tscrub1, 'C')
        process.getUnit('23-HA-03').setOutPressure(locinput.Psep3 - locinput.dP_20_HA_03, 'barg')

        # 23-PA-01
        process.getUnit('23-PA-01').setPressure(locinput.Psep3 + locinput.dP_20_HA_03, 'barg')

        # 23-KA-03
        process.getUnit('23-KA-03').setOutletPressure(locinput.Psep2, 'barg')

        # 23-HA-02
        process.getUnit('23-HA-02').setOutTemperature(locinput.Tscrub2, 'C')
        process.getUnit('23-HA-02').setOutPressure(locinput.Psep2 - locinput.dP_23_HA_02, 'barg')

        # 23-KA-02
        process.getUnit('23-KA-02').setOutletPressure(locinput.Psep1, 'barg')

        # 23-KA-01
        process.getUnit('23-KA-01').setOutletPressure(locinput.Pcomp1, 'barg')

        # 23-HA-01
        process.getUnit('23-HA-01').setOutTemperature(locinput.Tscrub3, 'C')
        process.getUnit('23-HA-01').setOutPressure(locinput.Psep1 - locinput.dP_23_HA_01, 'barg')

        # 24-HA-01
        process.getUnit('24-HA-01').setOutTemperature(locinput.Tscrub4, 'C')
        process.getUnit('24-HA-01').setOutPressure(locinput.Pcomp1 - locinput.dP_24_HA_01, 'barg')

        # 27-KA-01
        process.getUnit('27-KA-01').setOutletPressure(locinput.P_gas_export, 'barg')

        # 25-HA-02
        process.getUnit('25-HA-02').setOutTemperature(locinput.Trefig, 'C')
        process.getUnit('25-HA-02').setOutPressure(
            locinput.Pcomp1 - locinput.dP_25_HA_01 - locinput.dP_20_HA_02, 'barg'
        )

        # 21-HA-01
        process.getUnit('21-HA-01').setOutTemperature(locinput.T_oil_export, 'C')
        process.getUnit('21-HA-01').setOutPressure(
            locinput.Psep3 - locinput.dP_21_HA_01, 'barg'
        )

        # 21-PA-01
        process.getUnit('21-PA-01').setOutletPressure(locinput.P_oil_export + 1.01325)

        # 27-HA-01
        process.getUnit('27-HA-01').setOutTemperature(40.0, 'C')
        process.getUnit('27-HA-01').setOutletPressure(locinput.P_oil_export - locinput.dP_27_HA_01)

    except AttributeError as e:
        print(f"Failed to update unit parameters: {e}")


def getoutput(process):
    """
    Retrieve simulation results from the NEQSim process model.

    Generates a JSON report using NEQSim's built-in reporting functions,
    then calculates a simple mass balance based on the feed, fuel gas,
    export gas, and exported oil streams.

    Args:
        process: A NEQSim ProcessSystem object.

    Returns:
        dict: A dictionary containing:
            - 'results': The JSON-parsed NEQSim report.
            - 'mass_balance': The mass balance calculation (as a percentage).
    """
    # Generate a JSON report from the NEQSim process
    json_report = str(neqsim.process.util.report.Report(process).generateJsonReport())

    # Calculate a simple mass balance
    feed_flow = process.getUnit('well stream').getFlowRate('kg/hr')
    fuel_flow = process.getUnit('fuel gas').getFlowRate('kg/hr')
    export_gas_flow = process.getUnit('27-KA-01').getOutStream().getFlowRate('kg/hr')
    export_oil_flow = process.getUnit('20-VA-03').getOilOutStream().getFlowRate('kg/hr')

    mass_balance_calc = (feed_flow - fuel_flow - export_gas_flow - export_oil_flow) / feed_flow * 100

    return {
        'results': json.loads(json_report),
        'mass_balance': mass_balance_calc
    }


def run_simulation(process, input_params, timeout=60):
    """
    Run the oil process simulation with a specified timeout.

    - The input parameters are validated and applied to the process model.
    - The simulation is run in a separate thread, with a default timeout of 60 seconds.
    - If the thread times out, an interruption is triggered, and `None` is returned.

    Args:
        process: A NEQSim ProcessSystem object to run the simulation on.
        input_params (dict): A dictionary of parameters to initialize
            the ProcessInput data class.
        timeout (int, optional): Timeout in seconds. Defaults to 60.

    Returns:
        ProcessOutput or None:
            If the simulation completes successfully, returns a ProcessOutput
            object containing the simulation results. Otherwise, returns None.
    """
    # Validate and update process inputs
    updateinput(process=process, locinput=ProcessInput(**input_params))

    # Run the process in a separate thread
    thread = process.runAsThread()
    thread.join(timeout * 1000)  # Timeout in milliseconds

    # Check if the thread is still alive after the timeout and interupt/stop if alive
    if thread.isAlive():
        thread.interrupt()
        thread.join()
        print("Process calculation timed out. Consider recreating the process object or adjusting parameters.")
        return None

    # On successful completion, retrieve results and wrap in a ProcessOutput
    return ProcessOutput(**getoutput(process=process))

# Read data from field using Seeq

See example script:

In [ ]:
# Step 1: Import necessary libraries
# The 'spy' module is for interacting with Seeq, and 'pandas' is for data manipulation.
from seeq import spy
import pandas as pd

# If running outside of a Seeq Data Lab notebook, you would need to log in first.
# In Data Lab, you are typically already authenticated.
# spy.login(url='https://your-seeq-server.com', credentials_file='path/to/credentials.key')

# Step 2: Search for the signals of interest
# We search for items within a specific asset path that have 'Temperature' and 'Pressure' in their names.
# The search returns a DataFrame with metadata about the signals that match the criteria.
signals_to_find = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Name': ['Temperature', 'Pressure']
})

# Display the search results to verify we found the correct signals
print("--- Search Results ---")
display(signals_to_find)

# Step 3: Pull the data for a specified time range
# Define the start and end times for the data pull.
end_time = '2025-08-18'
start_time = '2025-08-11'

# The spy.pull() function retrieves the time-series data for the items found in the search.
# 'grid' specifies the time interval for the data points; Seeq will interpolate as needed.
# The result is a Pandas DataFrame where the index is the timestamp and columns are the signal values.
field_data_df = spy.pull(items=signals_to_find,
                         start=start_time,
                         end=end_time,
                         grid='1h') # Pull data at an hourly interval

# Step 4: Display the resulting DataFrame
print("\n--- Pulled Field Data ---")
display(field_data_df.head()) # Show the first 5 rows of the DataFrame
```

 Explanation of the Code

1.  **Import Libraries**: We import `spy` to communicate with the Seeq server and `pandas` to work with the data.
2.  **`spy.search()`**: This function queries your Seeq instance to find assets, signals, conditions, etc. You provide a dictionary specifying the properties to filter on, such as the `Path` to an asset and the `Name` of the signal. It returns a metadata DataFrame, which is useful for confirming you've found the right items before pulling a large amount of data.
3.  **`spy.pull()`**: This is the function that retrieves the actual time-series data.
    *   `items`: This argument takes the DataFrame of items returned by `spy.search()`.
    *   `start` and `end`: These define the time period for which you want to retrieve data.
    *   `grid`: This parameter is used to align the data to a uniform time grid (e.g., every hour, every 10 minutes). Seeq's engine handles the interpolation. If you want the raw, unprocessed data, you can set `grid=None`.
4.  **Result**: The output, `field_data_df`, is a standard Pandas DataFrame. This format makes it easy to perform any data manipulation, analysis, or visualization and to use it as input for other libraries like NeqSim.

# Update input from field data
We then update process parameters from field data

In [ ]:
inputparam = {
 'feed_rate': 8000.0,
 'Tsep1': field_data_df.loc[i, 'OSX-20-003X']+1.01325,
 'Tsep1': field_data_df.loc[i, 'OSX-T20-003X'],
 ....... and so on
}




In [ ]:
#we then run simulations
results = run_simulation(process=process1, input_params=inputparam)
# and read it into a dataframe
results_df = pd.DataFrame.from_dict(results, orient='index')

# Push historic data back to Seek

In [ ]:
# 1. Create the final DataFrame from your results log
final_results_df = results_df

# 2. ---> THIS IS THE CRITICAL FIX <---
# Set the 'Timestamp' column as the index of the DataFrame.
if 'Timestamp' in final_results_df.columns:
    final_results_df.set_index('Timestamp', inplace=True)
else:
    print("ERROR: 'Timestamp' column not found in the results DataFrame!")
    # Stop here if the timestamp column is missing
    raise KeyError("'Timestamp' column not found. Cannot set the index.")


# --- Now, continue with the push logic from before ---

# Select the columns you want to write to Seeq

data_to_push_df = final_results_df

# Create the metadata. The 'Name' for each item MUST EXACTLY MATCH
# the column name in 'data_to_push_df'.
metadata_list = [
    {'Name': 'sim_gas_export_rate_MSm3_day', 'Type': 'Signal', 'Value Unit Of Measure': 'MSm3/day'},
    {'Name': 'sim_oil_export_idSm3_hr', 'Type': 'Signal', 'Value Unit Of Measure': 'idSm3/hr'},
    # Add metadata for any other columns here
]
metadata_df = pd.DataFrame(metadata_list)

# Rename the columns to match the metadata
#data_to_push_df.columns = metadata_df['Name']

# Now, push to Seeq. This should now work correctly.
try:
    push_results = spy.push(
        data=data_to_push_df,
         #   metadata = metadata_df,
        workbook='Process Simulation Results',
        worksheet='Online Model started 08.08.2025'
    )
    print("\nSuccessfully pushed data to Seeq!")

except Exception as e:
    print(f"An error occurred during the push to Seeq: {e}")

The calculated data can then be visualized in Seeq workbooks together with signals and calculations from Seeq.

# Make a live process model (eg. self tuning process model)
A live process model can be made that wirtes pushes datat in real time back to seeq.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import seeq.spy as spy
import schedule
import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo  # Built-in in Python 3.9+


# === Main simulation + push function ===
def simulate_latest_point_from_seeq(osf_sture_model, ids):
    global texbypass_A, texbypass_B
    print("\n⏱ Running at:", datetime.now(ZoneInfo("Europe/Oslo")).isoformat() + 'Z')
    print("\n⏱ Running at:", datetime.now(ZoneInfo("Europe/Oslo")).isoformat() + 'Z')

    # 1. Define recent time window (UTC)
    end_time = datetime.now(ZoneInfo("Europe/Oslo"))
    start_time = end_time - timedelta(minutes=120)

    # Format correctly (ISO format with local offset, not 'Z')
    start_time_str = start_time.isoformat()
    end_time_str = end_time.isoformat()

    # 2. Pull data for all signals
    merged_df = pd.DataFrame()
    for i in ids:
        try:
            search_result = spy.search({"ID": i})
            if not search_result.empty:
                name = search_result['Name'].iloc[0]
                my_data = spy.pull(
                    search_result,
                    start=start_time_str,
                    end=end_time_str,
                    grid='5min',
                    header='Name'
                ).dropna()

                if not my_data.empty:
                    latest_row = my_data.iloc[[-1]].reset_index()
                    latest_row = latest_row.rename(columns={'index': 'Timestamp'})
                    latest_row = latest_row[['Timestamp', name]]
                    if merged_df.empty:
                        merged_df = latest_row
                    else:
                        merged_df = pd.merge(merged_df, latest_row, on='Timestamp', how='outer')
        except Exception as e:
            print(f"❌ Error pulling data for ID {i}: {e}")

    process_input = ProcessInput(
        firststagepressure_A=float(merged_df.loc[0, 'OSXX  -003A']) + 1.01325,
        firststagepressure_B=float(merged_df.loc[0, 'OS-XX']) + 1.01325,
        seccond_stage_pressure_A=float(merged_df.loc[0,'OSXX']) + 1.01325,
        ........
        ........
        here you can implement a self tuning model to mach measurment from field
        ....

# Push live data to Seeq

In [ ]:
    # 6. Convert to DataFrame and push to Seeq
    final_df = pd.DataFrame([sim_results])
    final_df.set_index('Timestamp', inplace=True)


    try:
        spy.push(
            data=final_df,
            workbook='Process Simulation Results - neqsim_model_base_2-c2minus03-c1plus03',
            worksheet='Sture Data'
        )
        print("📤 Successfully pushed results to Seeq.")
    except Exception as e:
        print(f"❌ Push to Seeq failed: {e}")

def scheduled_job():
    simulate_latest_point_from_seeq(osf_sture_model, ids)

# === Schedule to run every 1 minute ===
schedule.every(5).minutes.do(scheduled_job)

scheduled_job()
# === Main loop ===
print("⏳ Starting simulation loop. Ctrl+C to stop.")
while True:
    schedule.run_pending()
    time.sleep(1)